In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import geopandas as gpd
import matplotlib.gridspec as gridspec
from pathlib import Path
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import numpy as np
from scipy.spatial import cKDTree


def standardize_basin_name(name):
    """Standardize basin names by removing 'Jordan' and other variations"""
    if pd.isna(name):
        return name

    # Convert to string and strip whitespace
    name = str(name).strip()

    # Create mapping for common variations
    name_mappings = {
        'JORDAN VALLY (JORDAN)': 'JORDAN_VALLEY',
        'JORDAN_VALLEY': 'JORDAN_VALLEY',
        'N.R.S.W': 'NRSW',
        'NRSW': 'NRSW',
        'YARMOUK (JORDAN)': 'YARMOUK',
        'YARMOUK': 'YARMOUK',
        'AMMAN ZARQA (JORDAN)': 'AMMAN_ZARQA',
        'AMMAN_ZARQA': 'AMMAN_ZARQA',
        'AZRAQ (JORDAN)': 'AZRAQ',
        'AZRAQ': 'AZRAQ',
        'S.R.S.W': 'SRSW',
        'SRSW': 'SRSW',
        'D.S.R.S.W': 'DSRSW',
        'DSRSW': 'DSRSW',
        'W. ARABA NORTH': 'W_ARAB_NORTH',
        'W_ARAB_NORTH': 'W_ARAB_NORTH',
        'HAMMAD': 'HAMMAD',
        'MUJIB': 'MUJIB',
        'HASA': 'HASA',
        'JAFER': 'JAFER'
    }

    # Check if exact match exists
    if name in name_mappings:
        return name_mappings[name]

    # If no exact match, try removing variations of "Jordan" and convert to uppercase
    variations_to_remove = [
        ' (Jordan)', '(Jordan)', ' Jordan', 'Jordan ',
        ' (JORDAN)', '(JORDAN)', ' JORDAN', 'JORDAN '
    ]

    standardized = name
    for variation in variations_to_remove:
        if variation in standardized:
            standardized = standardized.replace(variation, '')

    # Clean up any extra whitespace and convert to uppercase
    standardized = standardized.strip().upper()

    return standardized


def load_basin_assignments():
    """Load and parse basin assignments from the provided data"""

    # Updated basin assignments using exact shapefile names and your Excel data
    assignments_data = {
        'HAMMAD': 'Single_Best_Model',
        'JORDAN VALLY (JORDAN)': 'Single_Best_Model',  # JORDAN_VALLEY from Excel
        'N.R.S.W': 'Single_Best_Model',  # NRSW from Excel
        'YARMOUK (JORDAN)': '3Model_Ensemble',  # YARMOUK from Excel
        'AMMAN ZARQA (JORDAN)': '6Model_Ensemble',  # AMMAN_ZARQA from Excel
        'MUJIB': 'Single_Best_Model',
        'AZRAQ (JORDAN)': 'Single_Best_Model',  # AZRAQ from Excel
        'S.R.S.W': 'Single_Best_Model',  # SRSW from Excel
        'D.S.R.S.W': '3Model_Ensemble',  # DSRSW from Excel
        'W. ARABA NORTH': 'Single_Best_Model',  # W_ARAB_NORTH from Excel
        'HASA': 'Single_Best_Model',
        'JAFER': 'Single_Best_Model'
    }

    # Convert to approach numbers
    approach_mapping = {
        'Single_Best_Model': 1,
        '3Model_Ensemble': 2,
        '6Model_Ensemble': 3
    }

    basin_approaches = {}
    for basin, approach_name in assignments_data.items():
        basin_approaches[basin] = {
            'approach': approach_mapping[approach_name],
            'approach_name': approach_name
        }

    return basin_approaches


def find_nearest_basin(target_basin, reference_basins, basin_approaches):
    """Find the nearest basin that has an approach assignment"""
    target_centroid = target_basin.geometry.centroid

    # Get centroids of reference basins
    ref_centroids = []
    ref_names = []

    for idx, basin in reference_basins.iterrows():
        standardized_name = standardize_basin_name(basin['BASIN_NAME'])
        if standardized_name in basin_approaches:
            centroid = basin.geometry.centroid
            ref_centroids.append([centroid.x, centroid.y])
            ref_names.append(standardized_name)

    if not ref_centroids:
        return None, float('inf')

    # Find nearest using KDTree
    tree = cKDTree(ref_centroids)
    distance, index = tree.query([target_centroid.x, target_centroid.y])

    return ref_names[index], distance


def assign_approaches_to_all_basins(basins_gdf, basin_approaches, output_dir):
    """Assign approaches to all basins, using nearest basin for missing assignments"""

    assignment_records = []
    final_assignments = {}

    for idx, basin in basins_gdf.iterrows():
        original_name = basin['BASIN_NAME']

        # Try direct match first
        if original_name in basin_approaches:
            # Direct assignment using exact shapefile name
            approach_info = basin_approaches[original_name]
            final_assignments[original_name] = approach_info['approach']

            assignment_records.append({
                'Basin_Original': original_name,
                'Basin_Standardized': original_name,
                'Assignment_Type': 'Direct',
                'Reference_Basin': original_name,
                'Distance_km': 0,
                'Approach_Number': approach_info['approach'],
                'Approach_Name': approach_info['approach_name']
            })
        else:
            # Try standardized name matching
            standardized_name = standardize_basin_name(original_name)

            # Look for standardized name in basin_approaches keys
            matched_key = None
            for key in basin_approaches.keys():
                if standardize_basin_name(key) == standardized_name:
                    matched_key = key
                    break

            if matched_key:
                approach_info = basin_approaches[matched_key]
                final_assignments[original_name] = approach_info['approach']

                assignment_records.append({
                    'Basin_Original': original_name,
                    'Basin_Standardized': standardized_name,
                    'Assignment_Type': 'Standardized_Match',
                    'Reference_Basin': matched_key,
                    'Distance_km': 0,
                    'Approach_Number': approach_info['approach'],
                    'Approach_Name': approach_info['approach_name']
                })
            else:
                # Find nearest basin with assignment
                nearest_basin, distance = find_nearest_basin(basin, basins_gdf, basin_approaches)

                if nearest_basin:
                    approach_info = basin_approaches[nearest_basin]
                    final_assignments[original_name] = approach_info['approach']

                    assignment_records.append({
                        'Basin_Original': original_name,
                        'Basin_Standardized': standardized_name,
                        'Assignment_Type': 'Nearest',
                        'Reference_Basin': nearest_basin,
                        'Distance_km': distance * 111,  # Convert degrees to km
                        'Approach_Number': approach_info['approach'],
                        'Approach_Name': approach_info['approach_name']
                    })
                else:
                    # Fallback to approach 1 (Single Best Model)
                    final_assignments[original_name] = 1
                    assignment_records.append({
                        'Basin_Original': original_name,
                        'Basin_Standardized': standardized_name,
                        'Assignment_Type': 'Fallback',
                        'Reference_Basin': 'None',
                        'Distance_km': 999,
                        'Approach_Number': 1,
                        'Approach_Name': 'Single_Best_Model'
                    })

    # Save assignment report
    assignments_df = pd.DataFrame(assignment_records)
    assignments_df.to_excel(output_dir / 'Basin_Approach_Assignments.xlsx', index=False)

    # Create summary statistics
    summary_stats = assignments_df.groupby(['Approach_Number', 'Approach_Name']).agg({
        'Basin_Original': 'count',
        'Assignment_Type': lambda x: (x == 'Direct').sum()
    }).round(2)
    summary_stats.columns = ['Total_Basins', 'Direct_Assignments']
    summary_stats['Nearest_Assignments'] = summary_stats['Total_Basins'] - summary_stats['Direct_Assignments']

    summary_stats.to_excel(output_dir / 'Approach_Summary_Statistics.xlsx')

    print(f"\nBasin Assignment Summary:")
    print(f"Approach 1 (Single Best Model): {len(assignments_df[assignments_df['Approach_Number'] == 1])} basins")
    print(f"Approach 2 (3-Model Ensemble): {len(assignments_df[assignments_df['Approach_Number'] == 2])} basins")
    print(f"Approach 3 (6-Model Ensemble): {len(assignments_df[assignments_df['Approach_Number'] == 3])} basins")

    # Print detailed assignments for verification
    print(f"\nDetailed Assignments:")
    for _, row in assignments_df.iterrows():
        print(f"  {row['Basin_Original']} -> Approach {row['Approach_Number']} ({row['Assignment_Type']})")

    return final_assignments, assignments_df


def get_approach_paths(approach_number):
    """Get file paths for each approach"""
    paths = {
        1: {  # Single Best Model
            'diff_dir': Path(
                r"D:\RICAAR\Pr.New.Stations.Selection\difference maps\mm.month.withncfiles8.5scenario\Difference_files"),
            'excel_dir': Path(r"D:\RICAAR\Pr.New.Stations.Selection\difference maps\mm.month.withncfiles8.5scenario\Excel_files")
        },
        2: {  # 3-Model Ensemble
            'diff_dir': Path(
                r"D:\RICAAR\Pr.New.Stations.Selection\ensemble.model8.5\nc.files\difference.maps\mm.month.withncfiles\Difference_files"),
            'excel_dir': Path(
                r"D:\RICAAR\Pr.New.Stations.Selection\ensemble.model8.5\nc.files\difference.maps\mm.month.withncfiles\Excel_files")
        },
        3: {  # 6-Model Ensemble
            'diff_dir': Path(
                r"D:\RICAAR\Pr.New.Stations.Selection\ensemble.model8.5\ensemble.6models\nc.files\difference.files\Excel_files\Difference_files"),
            'excel_dir': Path(
                r"D:\RICAAR\Pr.New.Stations.Selection\ensemble.model8.5\ensemble.6models\nc.files\difference.files\Excel_files")
        }
    }
    return paths[approach_number]


def setup_map_axes(ax):
    """Setup common map elements for each subplot"""
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 8}
    gl.ylabel_style = {'size': 8}

    ax.set_extent([35, 39.5, 29, 33.5], crs=ccrs.PlateCarree())


def create_grid_from_points(df, value_column):
    """Create grid arrays from point data"""
    lons = sorted(df['Longitude'].unique())
    lats = sorted(df['Latitude'].unique())

    dx = lons[1] - lons[0]
    dy = lats[1] - lats[0]

    lon_edges = np.append(lons - dx / 2, lons[-1] + dx / 2)
    lat_edges = np.append(lats - dy / 2, lats[-1] + dy / 2)

    grid_values = np.full((len(lats), len(lons)), np.nan)

    for _, row in df.iterrows():
        i = lats.index(row['Latitude'])
        j = lons.index(row['Longitude'])
        grid_values[i, j] = row[value_column]

    return lon_edges, lat_edges, grid_values


def load_hybrid_data_for_basin(basin_name, approach_number, season, comparison_type, basins_gdf):
    """Load precipitation data for a specific basin using its assigned approach"""

    approach_paths = get_approach_paths(approach_number)

    # Create basin mask
    basin_geometry = basins_gdf[basins_gdf['BASIN_NAME'] == basin_name].geometry.iloc[0]

    try:
        if approach_number == 3:  # 6-Model Ensemble - Use actual file patterns found
            if comparison_type == 'reference':
                file_path = approach_paths['excel_dir'] / f"precipitation_ssp85_ensemble_reference_{season}_season_mmpermonth.xlsx"
                sheet_name = 'Precipitation_Data'
            elif comparison_type == 'near_minus_ref':
                file_path = approach_paths['diff_dir'] / f"precipitation_ssp85_difference_{season}_near_minus_ref.xlsx"
                sheet_name = 'Difference_Data'
            elif comparison_type == 'mid_minus_ref':
                file_path = approach_paths['diff_dir'] / f"precipitation_ssp85_difference_{season}_mid_minus_ref.xlsx"
                sheet_name = 'Difference_Data'
            else:
                return None

        elif approach_number == 2:  # 3-Model Ensemble - Use actual file patterns found
            if comparison_type == 'reference':
                file_path = approach_paths['excel_dir'] / f"precipitation_rcp85_reference_{season}_season_mmpermonth.xlsx"
                sheet_name = 'Data'
            elif comparison_type == 'near_minus_ref':
                file_path = approach_paths['diff_dir'] / f"precipitation_rcp85_difference_{season}_near_minus_ref.xlsx"
                sheet_name = 'Data'
            elif comparison_type == 'mid_minus_ref':
                file_path = approach_paths['diff_dir'] / f"precipitation_rcp85_difference_{season}_mid_minus_ref.xlsx"
                sheet_name = 'Data'
            else:
                return None

        else:  # Approach 1 - Single Best Model - Use actual file patterns found
            if comparison_type == 'reference':
                file_path = approach_paths['excel_dir'] / f"precipitation_reference_{season}_season_mmpermonth.xlsx"
                sheet_name = 'Data'
            elif comparison_type == 'near_minus_ref':
                file_path = approach_paths['diff_dir'] / f"precipitation_difference_{season}_near_minus_ref.xlsx"
                sheet_name = 'Data'
            elif comparison_type == 'mid_minus_ref':
                file_path = approach_paths['diff_dir'] / f"precipitation_difference_{season}_mid_minus_ref.xlsx"
                sheet_name = 'Data'
            else:
                return None

        if file_path.exists():
            df = pd.read_excel(file_path, sheet_name=sheet_name)

            # Filter data points that fall within the basin geometry
            basin_points = []
            for _, row in df.iterrows():
                point = gpd.points_from_xy([row['Longitude']], [row['Latitude']])[0]
                if basin_geometry.contains(point) or basin_geometry.touches(point):
                    basin_points.append(row)

            if basin_points:
                return pd.DataFrame(basin_points)
            else:
                return None
        else:
            print(f"Warning: File not found - {file_path}")
            return None

    except Exception as e:
        print(f"Error loading data for basin {basin_name}, approach {approach_number}: {str(e)}")
        return None


def create_hybrid_approach_maps(basins_gdf, basin_assignments, output_dir):
    """Create maps showing hybrid approach results"""

    print("\nCreating hybrid approach precipitation maps...")

    # Create output directory
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    fig = plt.figure(figsize=(15, 10))
    gs = gridspec.GridSpec(2, 3, height_ratios=[1, 1], hspace=0.3, wspace=0.1)

    # Define standardized color scales for difference maps
    wet_diff_vmin, wet_diff_vmax = -8, 8  # Wet season scale
    dry_diff_vmin, dry_diff_vmax = -2, 3  # Dry season scale (adjusted for lower values)

    for season_idx, season in enumerate(['wet', 'dry']):
        print(f"\nProcessing {season} season hybrid maps...")

        # Collect all data for this season
        all_reference_data = []
        all_near_diff_data = []
        all_mid_diff_data = []

        basin_approach_summary = []

        for basin_name, approach_number in basin_assignments.items():
            print(f"  Loading {basin_name} (Approach {approach_number})...")

            # Load reference data
            ref_data = load_hybrid_data_for_basin(basin_name, approach_number, season, 'reference', basins_gdf)
            if ref_data is not None:
                ref_data['Basin'] = basin_name
                ref_data['Approach'] = approach_number
                all_reference_data.append(ref_data)

            # Load difference data
            near_diff = load_hybrid_data_for_basin(basin_name, approach_number, season, 'near_minus_ref', basins_gdf)
            if near_diff is not None:
                near_diff['Basin'] = basin_name
                near_diff['Approach'] = approach_number
                all_near_diff_data.append(near_diff)

            mid_diff = load_hybrid_data_for_basin(basin_name, approach_number, season, 'mid_minus_ref', basins_gdf)
            if mid_diff is not None:
                mid_diff['Basin'] = basin_name
                mid_diff['Approach'] = approach_number
                all_mid_diff_data.append(mid_diff)

            basin_approach_summary.append({
                'Basin': basin_name,
                'Approach': approach_number,
                'Season': season,
                'Reference_Points': len(ref_data) if ref_data is not None else 0,
                'Near_Diff_Points': len(near_diff) if near_diff is not None else 0,
                'Mid_Diff_Points': len(mid_diff) if mid_diff is not None else 0
            })

        # Save basin approach summary
        summary_df = pd.DataFrame(basin_approach_summary)
        summary_df.to_excel(output_dir / f'Basin_Data_Summary_{season}.xlsx', index=False)

        # Combine all data
        if all_reference_data:
            combined_ref = pd.concat(all_reference_data, ignore_index=True)

            # Reference period map
            ax0 = fig.add_subplot(gs[season_idx, 0], projection=ccrs.PlateCarree())
            setup_map_axes(ax0)

            # Create map for reference data
            value_col = 'Precipitation (mm/month)' if 'Precipitation (mm/month)' in combined_ref.columns else 'prAdjust'

            if value_col in combined_ref.columns:
                lon_edges, lat_edges, ref_values = create_grid_from_points(combined_ref, value_col)

                # Set reference scale based on season
                if season == 'wet':
                    vmin_ref, vmax_ref = 0, 70
                else:  # dry season
                    vmin_ref, vmax_ref = 0, 10

                im0 = ax0.pcolormesh(
                    lon_edges, lat_edges, ref_values,
                    cmap='Blues',
                    vmin=vmin_ref,
                    vmax=vmax_ref,
                    transform=ccrs.PlateCarree()
                )

                # Add basin boundaries with approach colors - thinner lines
                approach_colors = {1: 'red', 2: 'green', 3: 'blue'}
                for basin_name, approach in basin_assignments.items():
                    basin_geom = basins_gdf[basins_gdf['BASIN_NAME'] == basin_name]
                    if not basin_geom.empty:
                        basin_geom.boundary.plot(ax=ax0, edgecolor=approach_colors[approach],
                                                 linewidth=1.2, zorder=5)

                cbar0 = plt.colorbar(im0, ax=ax0, orientation='horizontal', pad=0.1, aspect=30)
                cbar0.set_label('Monthly Precipitation (mm/month)', fontsize=8)

                # Updated title format to match your image
                season_title = season.capitalize()
                ax0.set_title(f'{season_title} Season\nEnsemble Reference Period (1995-2014)',
                              pad=10, fontsize=10)

                # Add stats in bottom right
                ref_stats_text = (
                    f"Mean: {combined_ref[value_col].mean():.1f}\n"
                    f"Median: {combined_ref[value_col].median():.1f}\n"
                    f"Max: {combined_ref[value_col].max():.1f}\n"
                    f"Min: {combined_ref[value_col].min():.1f}"
                )
                ax0.text(0.98, 0.02, ref_stats_text,
                         transform=ax0.transAxes,
                         bbox=dict(facecolor='white', alpha=0.9, edgecolor='black'),
                         fontsize=8, ha='right', va='bottom')

        # Process difference maps with standardized scale
        diff_datasets = [
            (all_near_diff_data, '2021-2040', 1),
            (all_mid_diff_data, '2041-2060', 2)
        ]

        for diff_data_list, period, subplot_idx in diff_datasets:
            if diff_data_list:
                combined_diff = pd.concat(diff_data_list, ignore_index=True)

                ax = fig.add_subplot(gs[season_idx, subplot_idx], projection=ccrs.PlateCarree())
                setup_map_axes(ax)

                if 'Precipitation_Difference' in combined_diff.columns:
                    lon_edges, lat_edges, diff_values = create_grid_from_points(
                        combined_diff, 'Precipitation_Difference'
                    )

                    # Use season-specific standardized scale for difference maps
                    if season == 'wet':
                        diff_vmin, diff_vmax = wet_diff_vmin, wet_diff_vmax
                    else:  # dry season
                        diff_vmin, diff_vmax = dry_diff_vmin, dry_diff_vmax

                    im = ax.pcolormesh(
                        lon_edges, lat_edges, diff_values,
                        cmap='RdBu',
                        vmin=diff_vmin,
                        vmax=diff_vmax,
                        transform=ccrs.PlateCarree()
                    )

                    # Add basin boundaries with approach colors - thinner lines
                    for basin_name, approach in basin_assignments.items():
                        basin_geom = basins_gdf[basins_gdf['BASIN_NAME'] == basin_name]
                        if not basin_geom.empty:
                            basin_geom.boundary.plot(ax=ax, edgecolor=approach_colors[approach],
                                                     linewidth=1.2, zorder=5)

                    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.1, aspect=30)
                    cbar.set_label('Change in Monthly Precipitation (mm/month)', fontsize=8)

                    # Updated title format to match your image
                    season_title = season.capitalize()
                    ax.set_title(f'{season_title} Season\nEnsemble Change ({period})',
                                 pad=10, fontsize=10)

                    # Add stats in bottom right
                    diff_stats_text = (
                        f"Mean: {combined_diff['Precipitation_Difference'].mean():.1f}\n"
                        f"Median: {combined_diff['Precipitation_Difference'].median():.1f}\n"
                        f"Max: {combined_diff['Precipitation_Difference'].max():.1f}\n"
                        f"Min: {combined_diff['Precipitation_Difference'].min():.1f}"
                    )
                    ax.text(0.98, 0.02, diff_stats_text,
                            transform=ax.transAxes,
                            bbox=dict(facecolor='white', alpha=0.9, edgecolor='black'),
                            fontsize=8, ha='right', va='bottom')

    # Add legend for approaches at the bottom
    legend_elements = [
        plt.Line2D([0], [0], color='red', lw=3, label='Approach 1: Single Best Model'),
        plt.Line2D([0], [0], color='green', lw=3, label='Approach 2: 3-Model Ensemble'),
        plt.Line2D([0], [0], color='blue', lw=3, label='Approach 3: 6-Model Ensemble')
    ]
    fig.legend(handles=legend_elements, loc='lower center', bbox_to_anchor=(0.5, -0.02), ncol=3)

    # Updated main title to match your image
    fig.suptitle('Changes in Monthly Precipitation from Ensemble Model (SSP 8.5)',
                 y=0.95, fontsize=14)

    # Save the plot
    output_file = output_dir / 'hybrid_approach_precipitation_maps_ssp85.png'
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close()

    print(f"\nHybrid approach maps saved to: {output_file}")

    return output_file


def main():
    """Main function to create hybrid approach analysis"""

    # Define paths
    basin_shapefile = Path(r"D:\RICAAR\surfacebasin\surface_basin.shp")
    gov_shapefile = Path(r"D:\RICAAR\Governorates\JordanwithGovernorates.shp")
    output_dir = Path(r"D:\RICAAR\Pr.New.Stations.Selection\Hybrid approch\ssp8.5")

    print("=" * 80)
    print("HYBRID APPROACH ANALYSIS FOR OPTIMAL BASIN-SPECIFIC MODELS - SSP 8.5")
    print("=" * 80)
    print(f"Output directory: {output_dir}")

    # Create output directory
    output_dir.mkdir(parents=True, exist_ok=True)

    # Load shapefiles
    print("\nLoading shapefiles...")
    basins_gdf = gpd.read_file(basin_shapefile).to_crs(epsg=4326)
    jordan_gdf = gpd.read_file(gov_shapefile).to_crs(epsg=4326)

    # Filter basins - Remove Syrian basins (maintain original approach)
    print("\nFiltering basins (removing Syrian basins)...")
    exclude_basins = ['JVALLEYYARMOUKTRIANGLE']
    basins_gdf = basins_gdf[~basins_gdf['BASIN_NAME'].isin(exclude_basins)]
    basins_gdf = basins_gdf[~basins_gdf['BASIN_NAME'].str.contains("SYRIA", case=False, na=False)]

    print(f"Loaded {len(basins_gdf)} basins after filtering")

    # Load basin assignments
    print("\nLoading basin approach assignments...")
    basin_approaches = load_basin_assignments()

    # Assign approaches to all basins
    print("\nAssigning approaches to all basins...")
    basin_assignments, assignments_df = assign_approaches_to_all_basins(
        basins_gdf, basin_approaches, output_dir
    )

    # Create hybrid maps
    print("\nCreating hybrid approach maps...")
    map_file = create_hybrid_approach_maps(basins_gdf, basin_assignments, output_dir)

    # Create final summary report
    print("\nCreating final summary report...")

    summary_report = f"""
HYBRID APPROACH ANALYSIS SUMMARY - SSP 8.5
==========================================

Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}

METHODOLOGY:
-----------
- Approach 1: Single Best Model (9 basins)
- Approach 2: 3-Model Ensemble (2 basins) 
- Approach 3: 6-Model Ensemble (1 basin)
- Missing basins assigned to nearest basin's approach
- Syrian basins excluded (following original methodology)

BASIN ASSIGNMENTS:
---------------------------
Approach 1 (Single Best Model): HAMMAD, JORDAN_VALLEY, NRSW, MUJIB, AZRAQ, SRSW, W_ARAB_NORTH, HASA, JAFER
Approach 2 (3-Model Ensemble): YARMOUK, DSRSW  
Approach 3 (6-Model Ensemble): AMMAN_ZARQA

BASIN FILTERING:
---------------
- Excluded: JVALLEYYARMOUKTRIANGLE
- Excluded: All basins containing "SYRIA"
- Total basins processed: {len(basins_gdf)}

FILE NAMING PATTERNS USED:
--------------------------
Approach 1 (Single Best Model):
  Reference: precipitation_reference_{{season}}_season_mmpermonth.xlsx
  Difference: precipitation_difference_{{season}}_{{comparison}}.xlsx
  Sheet: Data

Approach 2 (3-Model Ensemble):
  Reference: precipitation_rcp85_reference_{{season}}_season_mmpermonth.xlsx
  Difference: precipitation_rcp85_difference_{{season}}_{{comparison}}.xlsx
  Sheet: Data

Approach 3 (6-Model Ensemble):
  Reference: precipitation_ssp85_ensemble_reference_{{season}}_season_mmpermonth.xlsx
  Difference: precipitation_ssp85_difference_{{season}}_{{comparison}}.xlsx
  Sheet: Precipitation_Data (ref) / Difference_Data (diff)

BASIN ASSIGNMENTS:
-----------------
{assignments_df.groupby('Approach_Number').size().to_string()}

DIRECT VS NEAREST ASSIGNMENTS:
-----------------------------
{assignments_df.groupby(['Approach_Number', 'Assignment_Type']).size().to_string()}

OUTPUT FILES:
------------
1. Basin_Approach_Assignments.xlsx - Detailed basin assignments
2. Approach_Summary_Statistics.xlsx - Summary statistics  
3. Basin_Data_Summary_wet.xlsx - Wet season data summary
4. Basin_Data_Summary_dry.xlsx - Dry season data summary
5. hybrid_approach_precipitation_maps_ssp85.png - Final maps

RESULTS:
--------
- Each basin uses its validated optimal model configuration
- Maps show precipitation projections using best approach per basin
- Color-coded basin boundaries indicate which approach was used
- Combined results provide most accurate regional projections
- Syrian basins excluded as per original methodology
"""

    with open(output_dir / 'Hybrid_Approach_Summary_Report.txt', 'w') as f:
        f.write(summary_report)

    print(f"\n" + "=" * 80)
    print("HYBRID APPROACH ANALYSIS COMPLETE!")
    print("=" * 80)
    print(f"All files saved to: {output_dir}")
    print(f"Main visualization: {map_file}")
    print(f"Assignment details: {output_dir / 'Basin_Approach_Assignments.xlsx'}")
    print(f"Basins processed: {len(basins_gdf)} (Syrian basins excluded)")


if __name__ == "__main__":
    main()

HYBRID APPROACH ANALYSIS FOR OPTIMAL BASIN-SPECIFIC MODELS
Output directory: D:\RICAAR\Pr.New.Stations.Selection\Hybrid approch

Loading shapefiles...

Filtering basins (removing Syrian basins)...
Loaded 16 basins after filtering

Loading basin approach assignments...

Assigning approaches to all basins...

Basin Assignment Summary:
Approach 1 (Single Best Model): 13 basins
Approach 2 (3-Model Ensemble): 2 basins
Approach 3 (6-Model Ensemble): 1 basins

Detailed Assignments:
  HAMMAD -> Approach 1 (Direct)
  YARMOUK (JORDAN) -> Approach 2 (Direct)
  J.VALLEY-YARMOUK TRIANGLE -> Approach 1 (Nearest)
  JORDAN VALLY (JORDAN) -> Approach 1 (Direct)
  N.R.S.W -> Approach 1 (Direct)
  AZRAQ (JORDAN) -> Approach 1 (Direct)
  AMMAN ZARQA (JORDAN) -> Approach 3 (Direct)
  S.R.S.W -> Approach 1 (Direct)
  MUJIB -> Approach 1 (Direct)
  D.S.R.S.W -> Approach 2 (Direct)
  W. ARABA NORTH -> Approach 1 (Direct)
  HASA -> Approach 1 (Direct)
  JAFER -> Approach 1 (Direct)
  WADI ARABA SOUTH -> Approa

NameError: name 'season' is not defined